In [36]:
import os
from PIL import Image, ExifTags # type: ignore
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count

# Constants to configure
INPUT_FOLDER = ""  # Replace with your input folder path
OUTPUT_FOLDER = ""   # Replace with your output folder path
ASPECT_RATIO = "4:3"  # Choose between "4:3" or "16:9"
# ASPECT_RATIO = "16:9"  # Choose between "4:3" or "16:9"
TARGET_WIDTH = 1280  # Set the target width for resizing
# MAX_WORKERS = min(32, cpu_count() * 2)  # Adjust based on workload and system
MAX_WORKERS = 16

print(f"Using {MAX_WORKERS} workers for image processing")



Using 16 workers for image processing


In [37]:
def print_exif_data(image_path):
    """
    Prints EXIF data of an image if it exists.

    Args:
        image_path (str): Path to the image file.
    """
    try:
        with Image.open(image_path) as img:
            exif_data = img._getexif()
            if exif_data:
                print(f"EXIF data for {image_path}:")
                for tag, value in exif_data.items():
                    tag_name = ExifTags.TAGS.get(tag, tag)
                    print(f"  {tag_name}: {value}")
            else:
                print(f"No EXIF data found for {image_path}.")
    except Exception as e:
        print(f"Error reading EXIF data from {image_path}: {e}")

In [38]:
def correct_orientation(image):
    try:
        exif = image._getexif()
        if exif:
            for tag, value in ExifTags.TAGS.items():
                if value == "Orientation":
                    orientation_tag = tag
                    break

            orientation = exif.get(orientation_tag)
            if orientation == 3:
                image = image.rotate(180, expand=True)
            elif orientation == 6:
                image = image.rotate(270, expand=True)
            elif orientation == 8:
                image = image.rotate(90, expand=True)
    except Exception as e:
        print(f"Error correcting orientation: {e}")
    return image

In [39]:
def process_image(filename, input_folder, output_folder, aspect_ratio, target_width):
    """
    Processes a single image: corrects orientation, removes metadata, resizes, and crops.
    """
    file_path = os.path.join(input_folder, filename)
    try:
        with Image.open(file_path) as img:
            # Correct orientation
            img = correct_orientation(img)

            # Remove metadata
            img_no_metadata = Image.new(img.mode, img.size)
            img_no_metadata.putdata(list(img.getdata()))

            # Get original dimensions and orientation
            original_width, original_height = img_no_metadata.size
            is_landscape = original_width >= original_height

            aspect_ratios = {"4:3": (4, 3), "16:9": (16, 9)}
            aspect_width, aspect_height = aspect_ratios[aspect_ratio]

            if is_landscape:
                target_height = int((target_width / aspect_width) * aspect_height)
            else:
                target_height = target_width
                target_width = int((target_height / aspect_height) * aspect_width)

            target_aspect = target_width / target_height
            original_aspect = original_width / original_height

            # Determine crop box to match target aspect ratio
            if original_aspect > target_aspect:
                new_width = int(original_height * target_aspect)
                left = (original_width - new_width) // 2
                top = 0
                right = left + new_width
                bottom = original_height
            else:
                new_height = int(original_width / target_aspect)
                left = 0
                top = (original_height - new_height) // 2
                right = original_width
                bottom = top + new_height

            # Crop and resize the image
            img_cropped = img_no_metadata.crop((left, top, right, bottom))
            img_resized = img_cropped.resize((target_width, target_height), Image.ANTIALIAS)

            # Save the resized image to the output folder
            output_path = os.path.join(output_folder, filename)
            img_resized.save(output_path, format="JPEG")

            print(f"Processed and saved: {output_path}")
            # Print EXIF data of the output file
            print_exif_data(output_path)
    except Exception as e:
        print(f"Skipping file {file_path}: {e}")

In [40]:
def resize_and_crop_images_multithreaded(input_folder, output_folder, aspect_ratio, target_width, max_workers):
    """
    Resizes and crops images in parallel using multithreading.
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for filename in files:
            executor.submit(process_image, filename, input_folder, output_folder, aspect_ratio, target_width)

# Run the multithreaded function
resize_and_crop_images_multithreaded(INPUT_FOLDER, OUTPUT_FOLDER, ASPECT_RATIO, TARGET_WIDTH, MAX_WORKERS)

Skipping file /Users/allanleung/Documents/Killarney Community Centre/Images/.DS_Store: cannot identify image file '/Users/allanleung/Documents/Killarney Community Centre/Images/.DS_Store'
Processed and saved: /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2309.JPG
Processed and saved: /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2311.JPGProcessed and saved: /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2304.JPG
No EXIF data found for /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2309.JPG.

Processed and saved: /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2298.JPG
No EXIF data found for /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2304.JPG.
No EXIF data found for /Users/allanleung/Documents/Killarney Community Centre/Images/Resize/IMG_2298.JPG.
No EXIF data found for /Users/allanleung/Documents/Killarney Community Centre/Imag